In [1]:
import numpy as np
import cv2

In [2]:
def region_of_interest(img, vertices, color3=(255,0,0), color1=255):
    mask = np.zeros_like(img)
    
    if len(img.shape) > 2:
      color = color3
    else:
      color = color1
        
    cv2.fillPoly(mask, vertices, color)
    masked_image = cv2.bitwise_and(img, mask)
    
    return masked_image

In [3]:
def get_fitline(img, f_lines):    
    lines = np.squeeze(f_lines)
    lines = lines.reshape(lines.shape[0]*2,2)
    rows,cols = img.shape[:2]
    output = cv2.fitLine(lines,cv2.DIST_L2,0, 0.01, 0.01)
    vx, vy, x, y = output[0], output[1], output[2], output[3]
    x1, y1 = int(((img.shape[0]-1)-y)/vy*vx + x) , img.shape[0]-1
    x2, y2 = int(((img.shape[0]/2+100)-y)/vy*vx + x) , int(img.shape[0]/2+100)
    
    result = [x1,y1,x2,y2]
    return result

In [4]:
cap = cv2.VideoCapture('test_videos/solidWhiteRight.mp4')
while(cap.isOpened()):
    ret,frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    kernel = 5;
    blur_gray = cv2.GaussianBlur(gray,(kernel,kernel),0)
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    rho = 2
    theta = np.pi/180
    threshold = 15
    min_line_length = 40
    max_line_gap = 20
    
    left_vertices = np.array([[(150,540),(480,300),(480,300),(485,540)]])
    right_vertices = np.array([[(480,540),(480,300),(480,300),(900,540)]])

    left_roi = region_of_interest(edges, left_vertices)
    right_roi = region_of_interest(edges, right_vertices)

    left_lines=cv2.HoughLinesP(left_roi, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
    right_lines=cv2.HoughLinesP(right_roi, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)

    L_line = get_fitline(left_roi,left_lines)
    R_line = get_fitline(right_roi,right_lines)
 
    vertice = np.array([[(L_line[0],L_line[1]),(L_line[2],L_line[3]),(R_line[2],R_line[3]),(R_line[0],R_line[1])]])
    result = region_of_interest(frame, vertice, (0,255,0))

    combo = cv2.addWeighted(result, 0.8, frame, 1, 0)

    cv2.imshow('frame',combo)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break;
        
cap.release()
cv2.destroyAllWindows()

error: OpenCV(3.4.4) /io/opencv/modules/imgproc/src/color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
